# trans effect of truncations in domains of PTEN in Endometrial for AKT Phosphoproteomics

This notebook creates a boxplot for the trans effect of PTEN truncations in the phosphatase domain and the C2 domain in Endometrial.

Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

In [2]:
# Step 1 - Create proteomics and truncations dataframe

def get_trunc_df(trans_gene, include_wt = False, gene='PTEN'):
    prot_and_mutations = en.join_omics_to_mutations(
        mutations_genes = [gene], omics_df_name = 'phosphoproteomics', omics_genes = trans_gene, 
        mutations_filter = ['Nonsense_Mutation','Frame_Shift_Del','Frame_Shift_Ins']) 
    prot_and_mutations = en.reduce_multiindex(prot_and_mutations, flatten=True)
    if include_wt == False:
        keep = ['Nonsense_Mutation','Frame_Shift_Del','Frame_Shift_Ins']
        in_keep = prot_and_mutations['PTEN_Mutation_'].isin(keep)
        trunc_mutations = prot_and_mutations[in_keep]
    else:
        keep = ['Nonsense_Mutation','Frame_Shift_Del','Frame_Shift_Ins','Wildtype_Tumor']
        in_keep = prot_and_mutations['PTEN_Mutation_'].isin(keep)
        trunc_mutations = prot_and_mutations[in_keep]
    print(trunc_mutations['PTEN_Mutation_'].value_counts())

    # Step 2 - Create binary column 
    trunc_mutations['Mutation'] = np.where(
                trunc_mutations[gene+'_Mutation_Status_'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')

    # Step 3 - Format the dataframe correctly for the t-test(just omics and binary columns for tumors)
    tumors = trunc_mutations[trunc_mutations.Sample_Status_ == 'Tumor'] #drop Normal samples
    columns_to_drop = [gene+"_Mutation_", gene+"_Mutation_Status_", "Sample_Status_"]
    e_trunc_wt = tumors.drop(columns_to_drop, axis = 1)
    return e_trunc_wt

In [3]:
def format_pval_annotation(pval_symbol, x1, x2, line_start = .05, line_height=.05):
    # for manual adjustment to pval annotations
    
    y, h = line_start, line_height
    plt.plot([x1, x1, x2, x2], #draw horizontal line
             [y, y+h, y+h, y], #vertical line
             lw=1.5, color= '.3')
    plt.text((x1+x2)*.5, # half between x coord
             y+h, pval_symbol, horizontalalignment='center', verticalalignment='bottom', color = "black")


# Step 1: Create the dfs with PTEN truncations mutation in different domains 

First, load in cancer data sets from cptac.

In [4]:
en = cptac.Endometrial()

# Create the df for the Phosphatase Domain

In [17]:
t_gene = 'AKT1'
t = get_trunc_df(t_gene) 

Nonsense_Mutation    25
Frame_Shift_Del      22
Frame_Shift_Ins       6
Name: PTEN_Mutation_, dtype: int64


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 69 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 6)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Create a df with only mutations located in the phosphatase domain. (Both domains effected by the mutation).

In [18]:
# create new column with starting amino acid (aa) as an int
t['aa'] = t.apply(lambda x: (x['PTEN_Location_']), axis=1)
df1 = t.replace({'aa': r'^p.[A-Z]*'}, {'aa': ''}, regex=True)
df2 = df1.replace({'aa': r'\*$'}, {'aa': ''}, regex=True)
df2 = df2.replace({'aa': r'[A-Z]fs\*\d*$'}, {'aa': ''}, regex=True)
df2['aa'] = pd.to_numeric(df2["aa"])

# Keep only mutations in the domain (cutoff aa = 185)
phosphatase = df2.loc[df2['aa'] <= 185]
phosphatase.head()

Name,AKT1_phosphoproteomics_S122,AKT1_phosphoproteomics_S124,AKT1_phosphoproteomics_S126,AKT1_phosphoproteomics_S129,AKT1_phosphoproteomics_T450,PTEN_Location_,Mutation,aa
Patient_ID,,,,,,,,
C3L-00032,NaN,0.21400,0.0505,0.03195,NaN,p.W111*,Truncation,111
C3L-00137,NaN,0.08425,-0.0201,0.08425,0.351,p.Y180*,Truncation,180
C3L-00156,NaN,-0.08310,-0.1341,-0.17400,0.258,p.E7*,Truncation,7
C3L-00586,NaN,-0.14100,-0.1410,-0.14100,NaN,p.E7*,Truncation,7
C3L-00601,NaN,0.02710,-0.2530,0.02710,NaN,p.E7*,Truncation,7


# Create the df for the C2 domain (binds PIP3)

In [19]:
t2 = get_trunc_df(t_gene) 

Nonsense_Mutation    25
Frame_Shift_Del      22
Frame_Shift_Ins       6
Name: PTEN_Mutation_, dtype: int64


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 69 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 6)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
# create new column with starting amino acid (aa) as an int
t2['aa'] = t2.apply(lambda x: (x['PTEN_Location_']), axis=1)
df1 = t2.replace({'aa': r'^p.[A-Z]*'}, {'aa': ''}, regex=True)
df2 = df1.replace({'aa': r'\*$'}, {'aa': ''}, regex=True)
df2 = df2.replace({'aa': r'[A-Z]fs\*\d*$'}, {'aa': ''}, regex=True)
df2['aa'] = pd.to_numeric(df2["aa"])

# Keep only mutations in the domain (cutoff aa = 190)
c2 = df2.loc[df2['aa'] >= 190]
c2.head()

Name,AKT1_phosphoproteomics_S122,AKT1_phosphoproteomics_S124,AKT1_phosphoproteomics_S126,AKT1_phosphoproteomics_S129,AKT1_phosphoproteomics_T450,PTEN_Location_,Mutation,aa
Patient_ID,,,,,,,,
C3L-00006,NaN,-0.04605,0.1481,-0.04605,NaN,p.R233*,Truncation,233
C3L-00145,NaN,-0.16100,-0.2980,0.17900,NaN,p.E242*,Truncation,242
C3L-00161,0.108,0.10800,-0.1155,-0.10700,NaN,p.K267Rfs*9,Truncation,267
C3L-00413,0.130,-0.13000,-0.1270,-0.13000,NaN,p.N262Qfs*35,Truncation,262
C3L-00563,NaN,-0.22400,0.0548,0.05480,NaN,p.Q214*,Truncation,214


# Get Wildtype_Tumor df

In [21]:
t3 = get_trunc_df(t_gene, include_wt=True) 

Nonsense_Mutation    25
Frame_Shift_Del      22
Wildtype_Tumor       20
Frame_Shift_Ins       6
Name: PTEN_Mutation_, dtype: int64


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 69 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 6)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
wt = t3.loc[t3['Mutation'] == 'Wildtype_Tumor']

# Append dfs

In [23]:
# Create Domain column
c2_df = c2.assign(Domain='C2_Truncation')
ph_df = phosphatase.assign(Domain='Phosphatase_Truncation')
wt_df = wt.assign(Domain='Wildtype_Tumor')

all_df = ph_df.append(c2_df)
all_df = all_df.append(wt_df)
all_df.head()

,AKT1_phosphoproteomics_S122,AKT1_phosphoproteomics_S124,AKT1_phosphoproteomics_S126,AKT1_phosphoproteomics_S129,AKT1_phosphoproteomics_T450,PTEN_Location_,Mutation,aa,Domain
Patient_ID,,,,,,,,,
C3L-00032,NaN,0.21400,0.0505,0.03195,NaN,p.W111*,Truncation,111.0,Phosphatase_Truncation
C3L-00137,NaN,0.08425,-0.0201,0.08425,0.351,p.Y180*,Truncation,180.0,Phosphatase_Truncation
C3L-00156,NaN,-0.08310,-0.1341,-0.17400,0.258,p.E7*,Truncation,7.0,Phosphatase_Truncation
C3L-00586,NaN,-0.14100,-0.1410,-0.14100,NaN,p.E7*,Truncation,7.0,Phosphatase_Truncation
C3L-00601,NaN,0.02710,-0.2530,0.02710,NaN,p.E7*,Truncation,7.0,Phosphatase_Truncation


# Step 2: T-test 

In [24]:
# T-test for phosphatase vs wt
phosphatase_wt = all_df.loc[all_df['Domain'] != 'C2_Truncation']
print(phosphatase_wt.Domain.unique())

# Keep sites with >= 10 measurements
phosphatase_wt = phosphatase_wt.dropna(axis = 'columns', thresh = 10)

phos_cols_df = phosphatase_wt.drop(columns=['PTEN_Location_', 'aa','Domain'])
cols = list(phos_cols_df.columns[:-1])

phos_pval = u.wrap_ttest(phosphatase_wt, 'Mutation', cols, correction_method = 'fdr_bh') #, return_all = True)
phos_pval

['Phosphatase_Truncation' 'Wildtype_Tumor']


In [25]:
# T-test for C2 vs wt
c2_wt = all_df.loc[all_df['Domain'] != 'Phosphatase_Truncation']
print(c2_wt.Domain.unique())

# Keep sites with >= 10 measurements
c2_wt = c2_wt.dropna(axis = 'columns', thresh = 10)

c2_cols_df = c2_wt.drop(columns=['PTEN_Location_', 'aa','Domain'])
cols = list(c2_cols_df.columns[:-1])

c2_pval = u.wrap_ttest(c2_wt, 'Mutation', cols, correction_method = 'fdr_bh')#, return_all = True)
c2_pval

['C2_Truncation' 'Wildtype_Tumor']


Keep the significant phosphosites

In [26]:
sites = []
if phos_pval is not None:
    sites = list(phos_pval.Comparison)
if c2_pval is not None:
    c2_sites = list(c2_pval.Comparison)
    for e in c2_sites:
        sites.append(e)
    

sig_df = all_df[sites]
sig_df = sig_df.rename(columns=lambda x: re.sub('_phosphoproteomics','', x))
sig_df = sig_df.join(all_df[['Mutation', 'Domain']])
plot_df = sig_df.melt(id_vars = ['Mutation','Domain'], var_name = 'Site', value_name = 'Omics')
plot_df

,Mutation,Domain,Site,Omics


# Plot

In [28]:
plt.rcParams['figure.figsize']=(15,10) #size of plot
sns.set(font_scale = 1.2)

boxplot = sns.boxplot(x='Site', y='Omics', data = plot_df, hue = 'Domain', 
                      hue_order = ['Wildtype_Tumor','Phosphatase_Truncation','C2_Truncation'], 
                      showfliers = False)    
boxplot.set_title('trans effect of PTEN Truncations in Both Domains on '+t_gene+' Phosphoproteomics')
boxplot = sns.stripplot(x='Site', y='Omics', data = plot_df, hue = 'Domain',
                        hue_order = ['Wildtype_Tumor','Phosphatase_Truncation','C2_Truncation'],
                        jitter = True, color = ".3", dodge = True)
boxplot.set(xlabel = "\nPTEN Wildtype/Truncations in Both Domains", ylabel = t_gene+' Phosphoproteomics')

# format legend
handles, labels = boxplot.get_legend_handles_labels()
plt.legend(handles[0:3], labels[0:3])

# Create significance symbols:
# * P ≤ 0.05   ** P ≤ 0.01   *** P ≤ 0.001 

#format_pval_annotation('ns', -.26, 0, 2.5)
#format_pval_annotation('***   p-val: 0.0008', -.26, .25, 3)


#plt.show()
plt.clf()
plt.close()

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\seaborn\categorical.py:410: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=-0.5, right=-0.5
  ax.set_xlim(-.5, len(self.plot_data) - .5, auto=None)
No handles with labels found to put in legend.
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\seaborn\categorical.py:410: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=-0.5, right=-0.5
  ax.set_xlim(-.5, len(self.plot_data) - .5, auto=None)
